# Map / Reduce en python avec mrjob

## Prérequis : installation de mrjob avec Anaconda

### Ajout de conda-forge comme canal de distribution

Dans le navigateur Anaconda, 

1. cliquer sur `Channels` puis `Add`.
2. Dans la boite de dialogue, saisir `conda-forge` puis valider.
3. Rafraichir l'index en cliquant sur le bouton `Update index...`. 

Alternativement, en ligne de commande, selon la [documentation](https://conda-forge.org/) :

```
conda config --add channels conda-forge
conda config --set channel_priority strict
conda install <package-name> 
```

### Installation de mr-job

Dans la boite de recherche à côté du bouton `Update index...` saisir `mrjob` puis valider.

Alternativement, en ligne de commande, saisir `conda install -c conda-forge mrjob`



In [4]:
%%file wordcount.py
from mrjob.job import MRJob

class ComptageDesMots(MRJob):

    def mapper(self, _, ligne):
        ligneNettoyee = ligne.translate(str.maketrans('', '', string.punctuation))
        listeDeMots=ligneNettoyee.split()
        for mot in listeDeMots:
            yield mot, 1
    
    def reducer(self, key, values):
        yield key, sum(values)

Writing wordcount.py


In [10]:
import wordcount
#reload(wordcount)

mr_job = wordcount.MRWordFrequencyCount(args=['Data/Le Corbeau et le Renard.txt'])
with mr_job.make_runner() as runner:
     runner.run()
     for key, value in mr_job.parse_output(runner.cat_output()):
        print(key, value)

No configs specified for inline runner


words 131
lines 18
chars 673
